# 2D CNN Aproach with scalogram

## Imports and setup

In [91]:
import csv
import os
from tqdm import tqdm
import random

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from ssqueezepy import cwt, icwt
from ssqueezepy.visuals import plot, imshow
from scipy import signal

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 100)
np.set_printoptions(formatter={'all':lambda x: str(x)})
pd.options.display.float_format = '{:.10f}'.format

## Functions

In [92]:
# Visualisation
class Plot:
    def __init__(self, df,  title):
        self.df = df
        self.fig = plt.figure(figsize =(25, 8)) 
        self.fig.suptitle(title) 

        self.ax = self.fig.add_axes([0.1, 0.1, 0.8, 0.8])
        max_xticks = 10
        xloc = plt.MaxNLocator(max_xticks)
        self.ax.xaxis.set_major_locator(xloc)
    def chart(self, column_names, anomalies = False, vertical = False):
        for column_name in column_names:
            self.ax.plot(self.df.index, self.df[column_name], label = column_name)
            if anomalies:
                self.ax.plot(self.df[self.df["Anomaly"]==True].index, self.df[column_name][self.df["Anomaly"]==True], marker = 'o', ms = 3, mec = 'r', mfc = 'r', linestyle = "None", label="Anomalies")
        if vertical:
            for i in range(0, len(self.df)-1): 
                if self.df["Anomaly"].iloc[i]:
                    self.ax.axvline(x = self.df.index[i], color = 'r', linestyle = 'dashed', linewidth=0.05)
        self.fig.autofmt_xdate(rotation=15)
        self.ax.legend()
    def show(self): 
        plt.show()
        
    def save(self, image_path):
        path = f'../images/{image_path}.png'
        self.fig.savefig(path, facecolor='white', transparent=False)
        plt.close(self.fig)

def create_save_sca(X, Y, step, row_length, parameter):
    for i in tqdm(range(0, len(X)), desc=f"Set creation progress..."): 
        fig, axs = plt.subplots(2)
        fig.suptitle('Chart and Scalogram')      
       
        Wx, scales = cwt(X[i], 'morlet')
        imshow(Wx,ticks=0, borders=0, abs=1, ax=axs[1], show=0)
        axs[0].plot(X[i])
        
        dir_path = f'Scalograms/S{step}L{row_length}/'
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
        fig.savefig(dir_path + f'{i+1}_sca.png', facecolor='white', transparent=False)
        plt.close(fig)
        
# Finding anomalies
def find_column_anomaly(df, column):
    anomaly = ((df[column] > df[column].mean()+10) | (df[column] < df[column].mean()-10))
    #anomaly[anomaly==0] = -1
    return anomaly


def find_anomalies_std(df, columns):
    anomalies = []
    for column in columns:
        anomalies_array = find_column_anomaly(df, column)
        anomalies.append(anomalies_array)
    df_anomalies = pd.DataFrame(anomalies).transpose().astype(int).sum(axis=1)
    df["Anomaly"] = df_anomalies>=0.75*len(columns)
    return df

# Data processing
def get_sets(df, columns, step, row_length):
    rows_num = int(len(df)/step - (row_length/step-1))
    
    values = df[columns].values
    anomalies = df['Anomaly'].values
 
    X = np.zeros((rows_num, row_length,len(columns)))
    Y = np.zeros((rows_num, 1))
    
    for i in range(0, rows_num):
        first_element = step*i
        last_element = step*i+row_length
        X[i] = values[first_element:last_element]
        Y[i] = anomalies[first_element:last_element].sum()
    Y = np.where(Y > 0, 1, 0)
    return X, Y

def wavelet_transformation(X, columns, values_type="Complex"):
    Wx_test, scales_test  = cwt(X[0][:,0], 'morlet')
    X_shape = Wx_test.shape
    scales_shape = scales_test.shape
    data = np.zeros((len(X), X_shape[0], X_shape[1], len(columns)*2))
    counter = 0
    for row in tqdm(X, desc=f"Data creation progress..."):
        images = []
        for i in range(0, len(row[0])):
            Wx, scales = cwt(row[:, i], 'morlet')
            real = np.reshape(Wx.real, (Wx.shape[0], Wx.shape[1], 1))
            imag = np.reshape(Wx.imag, (Wx.shape[0], Wx.shape[1], 1))
            image = np.concatenate([real, imag], axis=2)
            images.append(image)
        data[counter] = np.concatenate(images, axis=2)
        counter +=1
    return data


# Filw management
def create_directory(dataset_name):
    dir_path = f'datasets/{dataset_name}/'
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

## Processing data

In [93]:
df = pd.read_csv('../data/thm/processed_thermal_data.csv')

## Create fake test data

In [94]:
# Creating gradual increase
max_increase = 100
incline_temp = np.linspace(30, max_increase, len(df))
df["Incline_temp"] = incline_temp
df["B1_inclined"] = df["Incline_temp"] + df["B1_temp"]

# Creating SIN function
periods = 20
max_amplitude = 20
time        = np.linspace(0, periods*4, len(df));
amplitude   = np.sin(time)*max_amplitude
df["B1_sin"] = amplitude + df["B1_temp"]

# Creating SIN + gradual increase
df["B1_inclined_sin"] = df["B1_temp"] + df["Incline_temp"] + amplitude

# Creating Sigmoid
increase = 200
increase_gradient = 100
x = np.linspace(-increase_gradient, increase_gradient, len(df))
sigmoid = 1/(1 + np.exp(-x+40))*increase
df["B1_sigmoid"] = df["B1_temp"] + sigmoid 

increase = -100
increase_gradient = 50
x = np.linspace(-increase_gradient, increase_gradient, len(df))
sigmoid = 1/(1 + np.exp(-x+40))*increase
df["B2_inclined"] = df["B2_temp"] + incline_temp 
df["MB_sin"] = df["MB_temp"] - amplitude 
df["BMB_sigmoid"] = df["BMB_temp"] - sigmoid
df["B2_sigmoid"] = df["B2_temp"] + sigmoid 
df["MB_sigmoid"] = df["MB_temp"] + sigmoid 


## Preparing data for scalogram creation

In [98]:
step = 20
row_length = 40
test_size = 0.6
columns = ["B1_sigmoid", "B2_sigmoid","MB_sigmoid", "BMB_sigmoid"]

df = find_anomalies_std(df, ["B1_temp", "B2_temp","MB_temp", "BMB_temp"])
df_new = df.iloc[-100000:,:]

X, Y =  get_sets(df_new, columns, step, row_length)


data = wavelet_transformation(X, columns)
x_train, x_test, y_train, y_test = train_test_split(data, Y, test_size=test_size, shuffle=False) 

Data creation progress...: 100%|██████████| 4999/4999 [03:16<00:00, 25.44it/s]


## Preparing directories and saving scalograms

In [99]:
dataset_name = f'{"&".join(columns)}-{row_length}L{step}S{int(test_size*100)}T'
create_directory(dataset_name)

np.save(f'datasets/{dataset_name}/X.npy', X)
np.save(f'datasets/{dataset_name}/original_data.npy', df_new[columns+["Anomaly"]].values)
np.save(f'datasets/{dataset_name}/x_train.npy', x_train)
np.save(f'datasets/{dataset_name}/y_train.npy', y_train)
np.save(f'datasets/{dataset_name}/x_test.npy', x_test)
np.save(f'datasets/{dataset_name}/y_test.npy', y_test)

print("Save done")

Save done
